## OSRM (Open Source Routing Machine)

**Author**: SADDIK Imad
<br/>
**Date**: 30/12/2024

---

**Table of contents**<a id='toc0_'></a>    
- [Match service](#toc1_)    
  - [Loading the GPS data](#toc1_1_)    
  - [Plotting GPS data](#toc1_2_)    
  - [Using the service](#toc1_3_)    
    - [General options](#toc1_3_1_)    
    - [Additional options](#toc1_3_2_)    
    - [Making the request](#toc1_3_3_)    
      - [No radiuses](#toc1_3_3_1_)    
      - [With radiuses](#toc1_3_3_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

---

# <a id='toc1_'></a>[Match service](#toc0_)

Use this service if you want to snap GPS points to the road network in the most plausible way. Read more about it [here](https://project-osrm.org/docs/v5.24.0/api/?language=cURL#match-service).

## <a id='toc1_1_'></a>[Loading the GPS data](#toc0_)

Visite the [public GPS traces](https://www.openstreetmap.org/traces) and download a trace in `gpx` format. Make sure that the GPS trace is located inside the area you downloaded from `Geofrabrik`.

In [15]:
from pydantic import BaseModel


class Point(BaseModel):
    latitude: float
    longitude: float
    timestamp: int  # Unix timestamp

The `parse_gpx_and_extract_points` method parses a `GPX` file and extracts track points with their coordinates and timestamps.

In [ ]:
import xml.etree.ElementTree as ET
from datetime import datetime


def parse_gpx_and_extract_points(file_path: str) -> list[Point]:
    # Parse the XML file into a tree structure
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Define namespace mapping for GPX 1.1 schema
    ns = {"default": "http://www.topografix.com/GPX/1/1"}

    points = []
    for trkpt in root.findall(".//default:trkpt", ns):
        lat = float(trkpt.attrib["lat"])
        lon = float(trkpt.attrib["lon"])

        # Convert ISO 8601 timestamp (removing Z and adding UTC offset) to Unix timestamp
        time_str = trkpt.find("default:time", ns).text
        timestamp = int(datetime.fromisoformat(
            time_str.replace("Z", "+00:00")).timestamp())

        points.append(Point(latitude=lat, longitude=lon, timestamp=timestamp))

    return points


file_path = "../data/gps_trace.gpx"
points = parse_gpx_and_extract_points(file_path)
len(points)

1420

## <a id='toc1_2_'></a>[Plotting GPS data](#toc0_)

We have extracted 1420 points from the `GPX` file but the OSRM routing engine limits the number of points to use in a request to 100. We can change this limit to another value if we like and I will show you how to do this in a later notebook.

After displaying the points, we can see that some of them are outside the road. GPS traces are not 100% accurate because sometimes the signal is not as strong as it should be and creates these anomalies in the data. This is why we use the match service to correct these inconsistencies.

In [ ]:
import folium
from folium import Map


def get_folium_map(center_point: Point, points: list[Point], zoom_level: int = 14) -> Map:
    folium_map = folium.Map(
        location=[center_point.latitude, center_point.longitude], zoom_start=zoom_level)

    for point in points:
        folium.CircleMarker(
            location=[point.latitude, point.longitude],
            radius=3,
            color='blue',
            fill=True,
            fill_color='blue'
        ).add_to(folium_map)

    return folium_map


max_points_to_use = 100
folium_map = get_folium_map(
    center_point=points[0],
    points=points[:max_points_to_use],
    zoom_level=18
)
folium_map

## <a id='toc1_3_'></a>[Using the service](#toc0_)

### <a id='toc1_3_1_'></a>[General options](#toc0_)

The general options are used in every OSRM service, read more about them [here](https://project-osrm.org/docs/v5.5.1/api/?language=cURL#general-options). Here is a table that summarizes the options with their description.


| Parameter | Description |
|-----------|-------------|
| service | One of the following values: `route`, `nearest`, `table`, `match`, `trip`, `tile` |
| version | Version of the protocol implemented by the service. `v1` for all OSRM 5.x installations |
| profile | Mode of transportation, is determined statically by the Lua profile that is used to prepare the data using `osrm-extract`. Typically `car`, `bike` or `foot` if using one of the supplied profiles. |
| coordinates | String of format `{longitude},{latitude};{longitude},{latitude};{longitude},{latitude} ...]` or `polyline({polyline})`. |
| format | Only `json` is supported at the moment. This parameter is optional and defaults to `json`. |

### <a id='toc1_3_2_'></a>[Additional options](#toc0_)

The additional options that we can use in our requests to the match service are:
- `timestamps`: Timestamps for the input locations in seconds since UNIX epoch. Timestamps need to be monotonically increasing. They help OSRM infer the order of the data points.
- `radiuses`: Standard deviation of GPS precision used for map matching.

### <a id='toc1_3_3_'></a>[Making the request](#toc0_)

The match endpoint follows this format: `/match/v1/{profile}/{coordinates}?steps={true|false}&geometries={polyline|polyline6|geojson}&overview={simplified|full|false}&annotations={true|false}`.

#### <a id='toc1_3_3_1_'></a>[No radiuses](#toc0_)

Don't forget to limit the number of points used when making the request.

In [95]:
service = 'match'
version = 'v1'
profile = 'driving'
host = 'http://localhost:5000'

In [102]:
import requests

points_portion = points[:max_points_to_use]
coordinates = ";".join(
    [f"{point.longitude},{point.latitude}" for point in points_portion])
timestamps = ";".join([str(point.timestamp)
                       for point in points_portion])

url = f"{host}/{service}/{version}/{profile}/{coordinates}"
params = {
    "steps": "true",
    "geometries": "geojson",
    "overview": "full",
    "annotations": "true",
    "timestamps": timestamps
}

response = requests.get(url, params=params)
response.status_code

200

The snapped route can be extracted from the `matchings` field. You should get just one matching route out of that field.

In [103]:
snapped_route = response.json()
snapped_route.keys()

dict_keys(['code', 'matchings', 'tracepoints'])

This looks familiar, we have seen such an output in the first notebook where we talked about the `route` service. We will use the legs to draw the snapped route.

In [105]:
matched_route = snapped_route["matchings"][0]
matched_route.keys()

dict_keys(['confidence', 'geometry', 'legs', 'weight_name', 'weight', 'duration', 'distance'])

The snapped route looks interesting, in the far left, we can see that we did not make a sharp U turn to change the direction, instead OSRM found a way to change direction with no sharp turns. Overall, it did a great job in following the original GPS data.

In [ ]:
folium_map = get_folium_map(
    center_point=points[0],
    points=points_portion,
    zoom_level=18
)

colors = ["red", "black"]
legs = matched_route["legs"]
for i, leg in enumerate(legs):
    steps = leg["steps"]
    for step in steps:
        geometry = step["geometry"]
        coordinates = geometry["coordinates"]
        coordinates = [Point(latitude=lat, longitude=lon, timestamp=0)
                       for lon, lat in coordinates]

        color = colors[i % len(colors)]
        leg_popup = f"""Leg number: {i+1}
        <br>Distance: {step['distance']} m
        <br>Duration: {step['duration']} s
        """

        folium.PolyLine(
            locations=[[point.latitude, point.longitude]
                       for point in coordinates],
            color=color,
            popup=folium.Popup(leg_popup, max_width=300)
        ).add_to(folium_map)

folium_map

There is also a `confidence` value. If it is close to 1, it means that OSRM is confident that the matching route is as accurate as possible. We will see how to play with the optional options to alter that confidence value.

In [ ]:
matched_route["confidence"]

0.01001476

#### <a id='toc1_3_3_2_'></a>[With radiuses](#toc0_)

In [112]:
points_portion = points[:max_points_to_use]
coordinates = ";".join(
    [f"{point.longitude},{point.latitude}" for point in points_portion])
timestamps = ";".join([str(point.timestamp)
                       for point in points_portion])
radiuses = ";".join(["20"] * len(points_portion))

url = f"{host}/{service}/{version}/{profile}/{coordinates}"
params = {
    "steps": "true",
    "geometries": "geojson",
    "overview": "full",
    "annotations": "true",
    "timestamps": timestamps,
    "radiuses": radiuses
}

response = requests.get(url, params=params)
response.status_code

200

After increasing the radius for each data points from 5 (default value) to 20, we observe that the confidence score increased from 0.1 to 0.35.

In [113]:
snapped_route = response.json()
matched_route = snapped_route["matchings"][0]
matched_route["confidence"]

0.355131182

Changing the radiuses also affected the matched route. Play with that option until you find the result you like the most.

In [114]:
folium_map = get_folium_map(
    center_point=points[0],
    points=points_portion,
    zoom_level=18
)

colors = ["red", "black"]
legs = matched_route["legs"]
for i, leg in enumerate(legs):
    steps = leg["steps"]
    for step in steps:
        geometry = step["geometry"]
        coordinates = geometry["coordinates"]
        coordinates = [Point(latitude=lat, longitude=lon, timestamp=0)
                       for lon, lat in coordinates]

        color = colors[i % len(colors)]
        leg_popup = f"""Leg number: {i+1}
        <br>Distance: {step['distance']} m
        <br>Duration: {step['duration']} s
        """

        folium.PolyLine(
            locations=[[point.latitude, point.longitude]
                       for point in coordinates],
            color=color,
            popup=folium.Popup(leg_popup, max_width=300)
        ).add_to(folium_map)

folium_map